In [1]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
3205,application_1713270977862_3399,pyspark,idle,Link,Link,None,
3206,application_1713270977862_3400,pyspark,idle,Link,Link,None,
3207,application_1713270977862_3401,pyspark,idle,Link,Link,None,


In [2]:
import os
print(f"remote USER={os.getenv('USER',None)}")

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
3208,application_1713270977862_3402,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

remote USER=livy

In [3]:
# (prevent deprecated np.bool error since numpy 1.24, until a new version of pandas/Spark fixes this)
import numpy as np
np.bool = np.bool_

username=spark.conf.get('spark.executorEnv.USERNAME', 'anonymous')
hadoop_fs=spark.conf.get('spark.executorEnv.HADOOP_DEFAULT_FS','hdfs://iccluster067.iccluster.epfl.ch:8020')
print(f"remote username={username}\nhadoop_fs={hadoop_fs}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

remote username=sicheng
hadoop_fs=hdfs://iccluster067.iccluster.epfl.ch:8020

In [6]:
spark.sql(f'SHOW TABLES IN {username}').show(truncate = False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+--------------------------------+-----------+
|namespace|tableName                       |isTemporary|
+---------+--------------------------------+-----------+
|sicheng  |sbb_istdaten_latest             |false      |
|sicheng  |sbb_istdaten_latest_parquet     |false      |
|sicheng  |sbb_csv                         |false      |
|sicheng  |sbb_orc_stops                   |false      |
|sicheng  |sbb_stops_lausanne              |false      |
|sicheng  |geo_shapes                      |false      |
|sicheng  |sbb_orc_one_month               |false      |
|sicheng  |sbb_stops_lausanne_region       |false      |
|sicheng  |sbb_orc_2024_01                 |false      |
|sicheng  |sbb_stop_to_stop_lausanne_region|false      |
|sicheng  |sbb_orc_calendar                |false      |
|sicheng  |sbb_orc_stop_times              |false      |
|sicheng  |sbb_stop_times_lausanne_region  |false      |
|sicheng  |istdaten_examples               |false      |
|sicheng  |timetable           

In [8]:
timetable = spark.read.table(f'{username}.timetable')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
timetable.show(5, truncate = False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------------+-----------+--------------+------------+------+-------+---------+--------+------+--------+------+
|trip_id               |stop_id    |departure_time|arrival_time|monday|tuesday|wednesday|thursday|friday|saturday|sunday|
+----------------------+-----------+--------------+------------+------+-------+---------+--------+------+--------+------+
|1.TA.91-1-M-j24-1.73.H|8501120:0:7|04:50:00      |04:50:00    |FALSE |FALSE  |FALSE    |FALSE   |FALSE |FALSE   |TRUE  |
|1.TA.91-1-M-j24-1.73.H|8518452:0:3|04:53:00      |04:53:00    |FALSE |FALSE  |FALSE    |FALSE   |FALSE |FALSE   |TRUE  |
|1.TA.91-1-M-j24-1.73.H|8501118:0:1|04:56:00      |04:55:00    |FALSE |FALSE  |FALSE    |FALSE   |FALSE |FALSE   |TRUE  |
|1.TA.91-1-M-j24-1.73.H|8501117:0:3|04:58:00      |04:58:00    |FALSE |FALSE  |FALSE    |FALSE   |FALSE |FALSE   |TRUE  |
|1.TA.91-2-R-j24-1.1.H |8501120:0:4|05:35:00      |05:35:00    |TRUE  |TRUE   |TRUE     |TRUE    |TRUE  |FALSE   |FALSE |
+----------------------+

In [34]:
from pyspark.sql.window import Window
import pyspark.sql.functions as F

def estimate_delay(departure_time, stop_id_a, stop_id_b = None):

    
    return 1  

window_spec = Window.partitionBy("trip_id").orderBy("departure_time")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [68]:
@F.udf
def time_to_seconds(time_str):
    """Converts a time string to seconds in a day, handling None inputs."""
    if time_str is None:
        return None  # Return None if the input is None
    hours, minutes, seconds = map(int, time_str.split(':'))
    return hours * 3600 + minutes * 60 + seconds

@F.udf
def seconds_to_string(seconds):
    """Converts seconds in a day to a time string, handling None inputs."""
    if seconds is None:
        return None  # Return None if the input is None
    hours = seconds // 3600
    minutes = (seconds % 3600) // 60
    seconds = seconds % 60
    return f"{hours:02d}:{minutes:02d}:{seconds:02d}"

@F.udf
def estimate_delay(departure_time, stop_id_a, stop_id_b = None):  
    
    # ----------TODO---------
    # estimated delay minutes
    # -----------------------  
    
    return 0.5

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [66]:
def process_timetable(timetable: DataFrame, remove_terminal_stops = False) -> DataFrame:
    """
    Processes the timetable data to include next stop ID, next stop arrival time,
    and expected next stop arrival time. Optionally removes rows where the next stop
    arrival time is null.

    """
    # Set window for each trip_id with ascending order on departure_time
    window_spec = Window.partitionBy("trip_id").orderBy("departure_time")

    # Compute next stop ID and next stop arrival time
    timetable_with_next_stop = timetable.withColumn(
        "next_stop_id",
        F.lead("stop_id", 1).over(window_spec)
    ).withColumn(
        "next_stop_arrival_time",
        F.lead("arrival_time", 1).over(window_spec)
    )

    # Compute expected next stop arrival time
    timetable_with_expected_arrival = timetable_with_next_stop.withColumn(
        "expected_next_stop_arrival_time",
        F.when(
            F.col("next_stop_arrival_time").isNotNull(),
            seconds_to_string(
                (time_to_seconds("next_stop_arrival_time") + 
                 (60 * estimate_delay(F.col("departure_time"), F.col("stop_id"), F.col("next_stop_id")))).cast("integer")
            )
        )
    )

    # Optionally remove rows where the next stop arrival time is null
    if remove_terminal_stops:
        timetable_with_expected_arrival = timetable_with_expected_arrival.filter(
            F.col("next_stop_arrival_time").isNotNull()
        )

    return timetable_with_expected_arrival

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [69]:
# Example
process_timetable(timetable).select(
    "trip_id", "stop_id", "departure_time", "arrival_time", "next_stop_id",
    "next_stop_arrival_time", "expected_next_stop_arrival_time"
).show(10, truncate = False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------------+-----------+--------------+------------+------------+----------------------+-------------------------------+
|trip_id               |stop_id    |departure_time|arrival_time|next_stop_id|next_stop_arrival_time|expected_next_stop_arrival_time|
+----------------------+-----------+--------------+------------+------------+----------------------+-------------------------------+
|1.TA.91-2E-Y-j24-1.1.H|8501120:0:6|19:45:00      |19:45:00    |null        |null                  |null                           |
|1.TA.91-9-K-j24-1.1.H |8501120:0:4|25:07:00      |25:07:00    |null        |null                  |null                           |
|1.TA.91-DA-Y-j24-1.2.H|8501118:0:5|05:29:00      |05:28:00    |8501120:0:3 |05:35:00              |05:35:30                       |
|1.TA.91-DA-Y-j24-1.2.H|8501120:0:3|05:47:00      |05:35:00    |null        |null                  |null                           |
|1.TA.91-FH-Y-j24-1.1.H|8504010:0:2|23:57:00      |23:57:00    |85040